# Исследование надёжности заёмщиков

* Заказчик - кредитный отдел банка
* Цель - разобраться, влияет ли семейное положение и количество детей на факт своевременного погашения кредита:
    * Есть ли зависимость между наличием детей и своевременным возвратом кредита?
    * Есть ли зависимость между семейным положением и своевременным возвратом кредита?
    * Есть ли зависимость между уровнем дохода и своевременным возвратом кредита?
    * Как разные цели кредита влияют на его возврат в срок?

## Содержание
1. [Начало работы](#load_data)
2. [Предобработка данных](#preprocessing)
    * [Пропуски](#NA)
    * [Дубликаты](#duplicates)
    * [Категоризация](#categories)
    * [Артефакты](#artifacts)
3. [Анализ данных](#analysis)
    * [Зависимость между наличием детей и своевременным возвратом кредита](#task_1)
    * [Зависимость между семейным положением и своевременным возвратом кредита](#task_2)
    * [Зависимость между уровнем дохода и своевременным возвратом кредита](#task_3)
    * [Влияние целей кредита на его возврат](#task_4)
4. [Выводы](#conclusions)

## Начало работы <a id="load_data"></a>

Для начала импортируем нужные библиотеки и загрузим данные в *df* для дальнейшего изучения:

In [1]:
from pymystem3 import Mystem
from IPython.display import display
from collections import Counter
import pandas as pd

m = Mystem()

df = pd.read_csv('/datasets/data.csv')

Далее выведем информацию о таблице, чтобы составить дальшейший план действий:

In [2]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


Из пояснительного материала узнаём, что означает каждый столбец:
    <details>
- `children` - количество детей в семье
- `days_employed` - общий трудовой стаж в днях
- `dob_years` - возраст клиента в годах
- `education` - уровень образования клиента
- `education_id` - идентификатор уровня образования
- `family_status` - семейное положение
- `family_status_id` - идентификатор семейного положения
- `gender` - пол клиента
- `income_type` - типа занятости
- `debt` - имел ли задолженность по возврату кредитов
- `total_income` - ежемесячный доход
- `purpose` - цель получения кредита
    </details>
    
Как мы видим, с названиями столбцов всё впорядке, а вот дальше начинаются проблемы. Выведем первые 10 строк для наглядности:

In [3]:
display(df.head(10))

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


Теперь видны следующие проблемы:
   * столбцы *days_employed* и *total_income* имеют пропущенные значения
   * столбец *days_employed*, к тому же, имеет непонятные отрицательные значения
   * столбец *education* имеет не грубые повторы
   * столбцы *days_employed* и *total_income* имеют тип данных *float*, неудобные для работы

Облегчим таблицу, вынеся столбцы *education* и *family status* в соответствующие таблицы-словари, и  проверим таблицу на грубые повторы:

In [4]:
education_dict = df[['education_id', 'education']].copy()
family_status_dict = df[['family_status_id', 'family_status']].copy()

data = df[
    ['children', 'days_employed', 'dob_years', 'education_id', 'family_status_id', 
     'gender', 'income_type', 'debt', 'total_income', 'purpose']
]

print(f"Число дубликатов: {data.duplicated().sum()}")

Число дубликатов: 71


Теперь к списку проблем прибавляются ещё и дубликаты, но начнём по порядку.

___

## Предобработка данных
<a id='preprocessing'></a>
Сперва удалим дубликаты:

In [5]:
data = data.drop_duplicates()
print(f"Число дубликатов: {data.duplicated().sum()}")

Число дубликатов: 0


Далее займёмся пропусками.  
#### Пропуски 
<a id='NA'></a>
Узнаем их количество в обеих столбцах:

In [6]:
print(f"Количество пропусков в столбце 'days_employed' - {data['days_employed'].isna().sum()}")
print(f"Количество пропусков в столбце 'total_income' - {data['total_income'].isna().sum()}")

Количество пропусков в столбце 'days_employed' - 2103
Количество пропусков в столбце 'total_income' - 2103


Так как их количество равно, проверим зависимость:

In [7]:
data.loc[(data['days_employed'].isna())&(data['total_income'].isna())].shape[0] == data['days_employed'].isna().sum()

True

Посмотрим на сами данные:

In [8]:
data['days_employed'].head(10)

0     -8437.673028
1     -4024.803754
2     -5623.422610
3     -4124.747207
4    340266.072047
5      -926.185831
6     -2879.202052
7      -152.779569
8     -6929.865299
9     -2188.756445
Name: days_employed, dtype: float64

Посмотрим на медианные значения по типу занятости:

In [9]:
median_days = data.groupby('income_type')['days_employed'].median()
median_days

income_type
безработный        366413.652744
в декрете           -3296.759962
госслужащий         -2689.368353
компаньон           -1547.382223
пенсионер          365213.306266
предприниматель      -520.848083
сотрудник           -1574.202821
студент              -578.751554
Name: days_employed, dtype: float64

У безработных и пенсионеров по какой-то причине очень большие числа, а у остальных - отрицательные. Возможно, данные неправильно обрабатываются при выгрузке, или при занесении в базу. Так же по какой-то причие пропуски в столбцах *days_employed* и *total_income* совпадают (Возможно, с помощью этих данных идёт рассчёт дохода, или наоборот)
Так как этот столбец не особо важен для наших задач, заполним пропуски медианными значениями (чтобы исключить влияние выбросов) и приведём к типу `int`:

In [10]:
data['days_employed'] = data['days_employed'].abs()
median_days = data.groupby('income_type')['days_employed'].median()

for inc_type in median_days.index:
    data.loc[
        data['income_type'] == inc_type, 'days_employed'
    ] = data.loc[
            data['income_type'] == inc_type, 'days_employed'
    ].fillna(median_days[inc_type])

data['days_employed'] = data['days_employed'].astype('int')

print(f"Число пропусков в столбце 'days_employed': {data['days_employed'].isna().sum()}")
print(f"Тип данных: {data['days_employed'].dtype}")

Число пропусков в столбце 'days_employed': 0
Тип данных: int32


Так как столбец *total_income* важен для дальнейших рассчётов, заполним пропуски в нём средними значениями по типу занятости и тоже приведём к типу `int`:

In [11]:
mean_income = data.groupby('income_type')['total_income'].mean()

for inc_type in mean_income.index:
    data.loc[
        data['income_type'] == inc_type, 'total_income'
    ] = data.loc[
            data['income_type'] == inc_type, 'total_income'
    ].fillna(mean_income[inc_type])
    
data['total_income'] = data['total_income'].astype(int)

print(f"Число пропусков в столбце 'total_income': {data['total_income'].isna().sum()}")
print(f"Тип данных: {data['total_income'].dtype}")

Число пропусков в столбце 'total_income': 0
Тип данных: int32


Далее займёмся повторами.
___
#### Повторы
<a id='duplicates'></a>
В таблицах-словарях *education_dict* и *family_status_dict*.  
Сначала первый:

In [12]:
education_dict['education'].value_counts()

среднее                13750
высшее                  4718
СРЕДНЕЕ                  772
Среднее                  711
неоконченное высшее      668
ВЫСШЕЕ                   274
Высшее                   268
начальное                250
Неоконченное высшее       47
НЕОКОНЧЕННОЕ ВЫСШЕЕ       29
НАЧАЛЬНОЕ                 17
Начальное                 15
ученая степень             4
УЧЕНАЯ СТЕПЕНЬ             1
Ученая степень             1
Name: education, dtype: int64

Мы имеем большое количество повторов, вызванное разным написанием одних и тех же слов. Чтобы этого избежать, нужно при записи значений использовать `.lower()`  
Чтобы это исправить в выгруженных данных, нужно привести всю колонку к одному виду с помощью метода `str.lower()` и удалить дубликаты:

In [13]:
education_dict['education'] = education_dict['education'].str.lower()
education_dict = education_dict.drop_duplicates('education').reset_index(drop=True)
education_dict

,education_id,education
0,0,высшее
1,1,среднее
2,2,неоконченное высшее
3,3,начальное
4,4,ученая степень


Проделаем то же самое с другим словарём:

In [14]:
family_status_dict['family_status'].value_counts()

женат / замужем          12380
гражданский брак          4177
Не женат / не замужем     2813
в разводе                 1195
вдовец / вдова             960
Name: family_status, dtype: int64

Как видим, проблем с разным написанием эта таблица не имеет, поэтому просто удалим повторяющиеся записи:

In [15]:
family_status_dict = family_status_dict.drop_duplicates('family_status').reset_index(drop=True)
family_status_dict

,family_status_id,family_status
0,0,женат / замужем
1,1,гражданский брак
2,2,вдовец / вдова
3,3,в разводе
4,4,Не женат / не замужем


Теперь займёмся группировкой по целям кредита. 
___
#### Категоризация
<a id='categories'></a>
Посмотрим, какие вообще есть цели:

In [16]:
purpose = data['purpose'].drop_duplicates().reset_index(drop=True).to_frame()
purpose

,purpose
0,покупка жилья
1,приобретение автомобиля
2,дополнительное образование
3,сыграть свадьбу
4,операции с жильем
5,образование
6,на проведение свадьбы
7,покупка жилья для семьи
8,покупка недвижимости
9,покупка коммерческой недвижимости


Проведём лемматизацию и выясним основные цели кредита:

In [17]:
purpose['lem'] = purpose['purpose'].apply(m.lemmatize)

In [18]:
print(Counter(purpose['lem'].sum()))

Counter({' ': 59, '\n': 38, 'покупка': 10, 'недвижимость': 10, 'автомобиль': 9, 'образование': 9, 'жилье': 7, 'с': 5, 'операция': 4, 'на': 4, 'свой': 4, 'свадьба': 3, 'строительство': 3, 'получение': 3, 'высокий': 3, 'дополнительный': 2, 'для': 2, 'коммерческий': 2, 'жилой': 2, 'заниматься': 2, 'сделка': 2, 'приобретение': 1, 'сыграть': 1, 'проведение': 1, 'семья': 1, 'собственный': 1, 'подержать': 1, 'со': 1, 'подержанный': 1, 'профильный': 1, 'сдача': 1, 'ремонт': 1})


Как мы можем увидеть, хоть целей кредита и много (целых 38 штук), но их все можно разделить на несколько групп - 
* свадьба
* недвижимость
* автомобиль
* образование

Сделаем группировку:

In [19]:
category = {
    'свадьба': ['свадьба'],
    'недвижимость': ['жилье', 'недвижимость'],
    'автомобиль': ['автомобиль'],
    'образование': ['образование'],
}

def cat_pur(lem):

    for key in category:
        for word in category[key]:
            if word in lem:
                return key
    return 'другое'

purpose['purpose_category'] = purpose['lem'].apply(cat_pur)
purpose.drop('lem', axis='columns', inplace=True)
purpose['purpose_category'].value_counts()

недвижимость    17
автомобиль       9
образование      9
свадьба          3
Name: purpose_category, dtype: int64

In [20]:
purpose

,purpose,purpose_category
0,покупка жилья,недвижимость
1,приобретение автомобиля,автомобиль
2,дополнительное образование,образование
3,сыграть свадьбу,свадьба
4,операции с жильем,недвижимость
5,образование,образование
6,на проведение свадьбы,свадьба
7,покупка жилья для семьи,недвижимость
8,покупка недвижимости,недвижимость
9,покупка коммерческой недвижимости,недвижимость


Заменим в *data* колонку *purpose* на *purpose_category*:

In [21]:
data = data.merge(purpose, on='purpose', how='left')
data.drop('purpose', axis='columns', inplace=True)

In [22]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21454 entries, 0 to 21453
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   children          21454 non-null  int64 
 1   days_employed     21454 non-null  int32 
 2   dob_years         21454 non-null  int64 
 3   education_id      21454 non-null  int64 
 4   family_status_id  21454 non-null  int64 
 5   gender            21454 non-null  object
 6   income_type       21454 non-null  object
 7   debt              21454 non-null  int64 
 8   total_income      21454 non-null  int32 
 9   purpose_category  21454 non-null  object
dtypes: int32(2), int64(5), object(3)
memory usage: 1.6+ MB


Осталось "причесать" нашу таблицу.
___
#### Артефакты
<a id='artifacts'></a>
Проверим столбцы на артефакты и исправим их.  
Проверим *children*:

In [23]:
data['children'].value_counts()

 0     14091
 1      4808
 2      2052
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

Мы наблюдаем 2 артефакта: значение *20* в 76 случаях и отрицательное значение в 47 случаях. Скорее всего, оба артефакта вызваны пробемой с валидацией полей при заполнении анкеты (в первом случае, вероятно, так обрабатывается пробел, а во втором - необработаная ошибка). Заменим оба артефакта на наиболее вероятные *0* детей:

In [24]:
data.loc[data['children'] == -1, 'children'] = 0
data.loc[data['children'] == 20, 'children'] = 0
data['children'].value_counts()

0    14214
1     4808
2     2052
3      330
4       41
5        9
Name: children, dtype: int64

Проверим `dob_years`:

In [25]:
data['dob_years'].unique()

array([42, 36, 33, 32, 53, 27, 43, 50, 35, 41, 40, 65, 54, 56, 26, 48, 24,
       21, 57, 67, 28, 63, 62, 47, 34, 68, 25, 31, 30, 20, 49, 37, 45, 61,
       64, 44, 52, 46, 23, 38, 39, 51,  0, 59, 29, 60, 55, 58, 71, 22, 73,
       66, 69, 19, 72, 70, 74, 75], dtype=int64)

На первый взгляд всё нормально, но есть нулевое значение. Это артефакт, следует рассмотреть поближе:

In [26]:
data.loc[data['dob_years'] == 0].head(10)

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose_category
99,0,346541,0,1,0,F,пенсионер,0,71291,автомобиль
149,0,2664,0,1,3,F,сотрудник,0,70176,недвижимость
270,3,1872,0,1,0,F,сотрудник,0,102166,недвижимость
578,0,397856,0,1,0,F,пенсионер,0,97620,недвижимость
1040,0,1158,0,0,3,F,компаньон,0,303994,автомобиль
1149,0,934,0,1,0,F,компаньон,0,201852,недвижимость
1175,0,370879,0,1,0,F,пенсионер,0,313949,образование
1386,0,5043,0,0,0,M,госслужащий,0,240523,автомобиль
1890,0,1574,0,0,4,F,сотрудник,0,161380,недвижимость
1898,0,370144,0,1,2,F,пенсионер,0,127400,автомобиль


Так как столбец не особо важен для наших задач и нет видимых паттернов возникновения, заменим нулевые значения на медианные по типу занятости:

In [27]:
median_years = data.groupby('income_type')['dob_years'].median().astype(int)

for inc_type in median_years.index:
    data.loc[(data['dob_years'] == 0)&(data['income_type'] == inc_type), 'dob_years'] = median_years[inc_type]
    
data['dob_years'].unique()

array([42, 36, 33, 32, 53, 27, 43, 50, 35, 41, 40, 65, 54, 56, 26, 48, 24,
       21, 57, 67, 28, 63, 62, 47, 34, 68, 25, 31, 30, 20, 49, 37, 45, 61,
       64, 44, 52, 46, 23, 38, 39, 51, 60, 59, 29, 55, 58, 71, 22, 73, 66,
       69, 19, 72, 70, 74, 75], dtype=int64)

От нулей избавились.  
Проверим *gender*:

In [28]:
data['gender'].value_counts()

F      14174
M       7279
XNA        1
Name: gender, dtype: int64

Видим один странный артефакт (возможно, так обработалась какая-то ошибка). Удалим одну строку:

In [29]:
data.drop(data[data['gender'] == 'XNA'].index, inplace=True)
data['gender'].unique()

array(['F', 'M'], dtype=object)

Остались только *F* и *M* (которые можно было бы заменить для удобства на *M* и *Ж*, но столбец особой роли не играет, поэтому оставим так).  
Проверим оставшийся *debt*:

In [30]:
data['debt'].value_counts()

0    19712
1     1741
Name: debt, dtype: int64

В итоге после всех действий имеем такую таблицу:

In [31]:
data.head(10)

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose_category
0,1,8437,42,0,0,F,сотрудник,0,253875,недвижимость
1,1,4024,36,1,0,F,сотрудник,0,112080,автомобиль
2,0,5623,33,1,0,M,сотрудник,0,145885,недвижимость
3,3,4124,32,1,0,M,сотрудник,0,267628,образование
4,0,340266,53,1,1,F,пенсионер,0,158616,свадьба
5,0,926,27,0,1,M,компаньон,0,255763,недвижимость
6,0,2879,43,0,0,F,компаньон,0,240525,недвижимость
7,0,152,50,1,0,M,сотрудник,0,135823,образование
8,2,6929,35,0,1,F,сотрудник,0,95856,свадьба
9,0,2188,41,1,0,M,сотрудник,0,144425,недвижимость


In [32]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21453 entries, 0 to 21453
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   children          21453 non-null  int64 
 1   days_employed     21453 non-null  int32 
 2   dob_years         21453 non-null  int64 
 3   education_id      21453 non-null  int64 
 4   family_status_id  21453 non-null  int64 
 5   gender            21453 non-null  object
 6   income_type       21453 non-null  object
 7   debt              21453 non-null  int64 
 8   total_income      21453 non-null  int32 
 9   purpose_category  21453 non-null  object
dtypes: int32(2), int64(5), object(3)
memory usage: 1.6+ MB


Настало время ответить на поставленные вопросы.
___ ___
## Анализ данных
<a id='analysis'></a>
Перед нами стоит 4 задачи. Рассмотрим их по порядку.

#### Зависимость между наличием детей и своевременным возвратом кредита
<a id='task_1'></a>



Создадим таблицу *children_data*, которую будем анализировать, и столбец *is_child* со значениями *0* и *1*, в зависимости от наличия детей:

In [33]:
children_data = data[['children', 'debt']].copy()
children_data['is_child'] = children_data['children']
children_data.loc[children_data['children'] > 1, 'is_child'] = 1

Далее создадим таблицу *child_pivot* и рассчитаем процент должников в двух категориях - *без детей* и *с детьми*:

In [34]:
child_pivot = children_data.pivot_table(index='is_child', columns='debt', values='children', aggfunc='count')
child_pivot.columns = ['Нет долга', 'Есть долг']
child_pivot['% должников'] = (child_pivot['Есть долг'] /
                             (child_pivot['Есть долг']
                             + child_pivot['Нет долга'])).map('{:.2%}'.format)
child_pivot

,Нет долга,Есть долг,% должников
is_child,,,
0,13141,1072,7.54%
1,6571,669,9.24%


Как мы видим, наличие детей **увеличивает** процент должников почти на 2.  
Взглянем на такую же таблицу, но по количеству детей:

In [35]:
children_pivot = children_data.pivot_table(index='children', columns='debt', values='is_child', aggfunc='count')
children_pivot.columns = ['Нет долга', 'Есть долг']
children_pivot['% должников'] = (children_pivot['Есть долг'] /
                                       (children_pivot['Есть долг']
                                       + children_pivot['Нет долга'])).map('{:.2%}'.format)
children_pivot[(children_pivot['Нет долга'] + children_pivot['Есть долг']) > 1000].sort_values('% должников')

,Нет долга,Есть долг,% должников
children,,,
0,13141.0,1072.0,7.54%
1,4364.0,444.0,9.23%
2,1858.0,194.0,9.45%


Мы видим более детальную картину - с увеличением количества детей растёт и количество должников.  
С дальнейшим увеличением количества детей уменьшается репрезентативность выборки и, следовательно, падает достоверность результатов, поэтому особо доверять этим значениям я бы не стал (нужно больше данных для конкретных выводов по количеству детей).

___
#### Зависимость между семейным положением и своевременным возвратом кредита
<a id='task_2'></a>

Создадим таблицу *family_data* и, для удобства, добавим значения из словаря *family_status_dict*:

In [36]:
family_data = data[['family_status_id', 'debt']]
family_data = family_data.merge(family_status_dict, on='family_status_id', how='left')

Далее на основе только что созданной таблицы создадим сводную таблицу `family_pivot` и рассчитаем процент должников в зависимости от семейного статуса:

In [37]:
family_pivot = family_data.pivot_table(index='family_status', columns='debt', values='family_status_id', aggfunc='count')
family_pivot.columns = ['Нет долга', 'Есть долг']
family_pivot['% должников'] = (family_pivot['Есть долг'] /
                                     (family_pivot['Есть долг']
                                     + family_pivot['Нет долга'])).map('{:.2%}'.format)
family_pivot.sort_values('% должников')

,Нет долга,Есть долг,% должников
family_status,,,
вдовец / вдова,896,63,6.57%
в разводе,1110,85,7.11%
женат / замужем,11408,931,7.55%
гражданский брак,3762,388,9.35%
Не женат / не замужем,2536,274,9.75%


Мы может увидеть, что у людей, потерявших супруга/супругу, самый низкий процент невозвратов, а самый высокий (с разницей почти в 3%) - среди тех, кто не в браке.  
Возможно, дело в возрасте:

In [38]:
family_data['age'] = data['dob_years'].copy()
family_data.groupby('family_status')['age'].median().sort_values()

family_status
Не женат / не замужем    39.0
гражданский брак         42.0
женат / замужем          43.0
в разводе                45.0
вдовец / вдова           53.0
Name: age, dtype: float64

Как мы видим, медианный возраст между этим двумя категориями отличается более чем на 10 лет.

In [39]:
family_data.groupby('family_status')['age'].median().sort_values(ascending=False).index == family_pivot.sort_values('% должников').index

array([ True,  True,  True,  True,  True])

Более того, наблюдается корелляция между семейным положением и возрастом, из чего можно сделать выводы, что на шанс невозврата кредита в срок влияет так же и возраст (с увеличением возраста уменьшается шанс невозврата)

___
#### Зависимость между уровнем дохода и своевременным возвратом кредита
<a id='task_3'></a>

Создадим таблицу *income_data* и добавим столбец *income_category* с помощью метода `.qcut()`, чтобы получить равномерное распределение по группам:

In [40]:
income_data = data[['total_income', 'debt']].copy()
income_data['income_category'] = pd.qcut(income_data['total_income'], 5, [
                                        'низкий', 'ниже среднего', 'средний', 'выше среднего', 'высокий'])

Далее создадим сводную таблицу *income_pivot* и рассчитаем процент должников в зависимости от уровня дохода:

In [41]:
income_pivot = income_data.pivot_table(index='income_category', columns='debt', values='total_income', aggfunc='count')
income_pivot.columns = ['Нет долга', 'Есть долг']
income_pivot['% должников'] = (income_pivot['Есть долг'] /
                              (income_pivot['Есть долг']
                              + income_pivot['Нет долга'])).map('{:.2%}'.format)
income_pivot.sort_values('% должников')

,Нет долга,Есть долг,% должников
income_category,,,
высокий,3991,300,6.99%
низкий,3947,344,8.02%
ниже среднего,3935,355,8.28%
выше среднего,3932,358,8.34%
средний,3907,384,8.95%


Меньше всех должников в категории с высоким доходом, а у категории со средним доходом процент должников выше на 2%. Для конкретизации не хватает данных.

___
#### Влияние целей кредита на его возврат
<a id='task_4'></a>

Так как дополнительной информации в этом случае не нужно, сразу создадим сводную таблицу *purpose_pivot* и рассчитаем процент должников в зависимости от целей кредита:

In [42]:
purpose_pivot = data.pivot_table(index='purpose_category', columns='debt', values='dob_years', aggfunc='count')
purpose_pivot.columns = ['Нет долга', 'Есть долг']
purpose_pivot['% должников'] = (purpose_pivot['Есть долг'] /
                                (purpose_pivot['Есть долг']
                                + purpose_pivot['Нет долга'])).map('{:.2%}'.format)
purpose_pivot.sort_values('% должников')

,Нет долга,Есть долг,% должников
purpose_category,,,
недвижимость,10028,782,7.23%
свадьба,2138,186,8.00%
образование,3643,370,9.22%
автомобиль,3903,403,9.36%


Как мы видим, на первом месте по невозвратам - кредиты на автомобиль. Возможно, это связано с повышеной опасностью на дорогах.  
Для исключения ошибок проверим, сходится ли общая сумма заёмщиков в каждом исследовании:

In [43]:
(child_pivot['Есть долг'].sum() + child_pivot['Нет долга'].sum()) == (
    children_pivot['Есть долг'].sum() + children_pivot['Нет долга'].sum()) ==(
    family_pivot['Есть долг'].sum() + family_pivot['Нет долга'].sum()) == (
    income_pivot['Есть долг'].sum() + income_pivot['Нет долга'].sum())  ==(
    purpose_pivot['Есть долг'].sum() + purpose_pivot['Нет долга'].sum())

True

Всё сходится. Перейдём к выводам.

## Выводы
<a id='conclusions'></a>

Проведя исследование, мы выяснили:
- Наличие детей у заёмщика влияет на шанс невозврата кредита в срок, и с ростом количества детей этот шанс может увеличиваться.
- Что на этот шанс влияет так же и семейное положение заёмщика (а вкупе с возрастом можно делать более точные выводы). 
- Уровень дохода оказывает влияние, но недостаточно данных для обоснования этого влияния. Пока что можно сделать выводы, что если уровень дохода заёмщика попадает в категорию "высокий", то шансы невозврата кредита в срок минимальны по сравнению с другими категориями
- Цели заёма так же влияют на шанс несвоевременного возврата кредита, и на основе имеющихся данных можно сделать вывод, что заёмы на автомобили являются самыми рискованными из рассматриваемых категорий

Помимо ответов на поставленные вопросы мы обнаружили некоторые технические проблемы:
- Вероятно, имеет место быть проблема с валидацией данных и обработкой ошибок поля *children*
- Так же, вероятно, есть ошибка в обработке поля *gender*
- Поле *education* нужно приводить к одному регистру (а желательно вообще сделать выбор из списка, чтобы не было никаких проблем)
- Поля *dob_years* и *total_income* по непонятной причине имеют пропуски в одних и тех же строках, к тому же поле *dob_years*, скорее всего, неправильно обрабатывается или выгружается
- Категоризацию поля *purpose* тоже можно сделать при обработке это поля в анкете, чтобы не тратить время на лемматизацию большого объёма данных при анализе данных